![Insper](https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Insper-Logo.png?raw=true)

# Insper Pós-Graduação
## Programa Avançado em Data Science e Decisão [»](https://www.insper.edu.br/pos-graduacao/programas-avancados/programa-avancado-em-data-science-e-decisao/)


# Atividade Integradora
## Setup

### Dependências

In [ ]:
import pandas as pd
import numpy as np
from dfply import *
import altair as alt
import missingno as msno
from ydata_profiling import ProfileReport
import matplotlib
import matplotlib.pyplot as plt 
import math
import seaborn as sns
from sklearn import linear_model
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

### Carregamento dos Dados

Leitura da base de dados e do dicionários de dados

In [ ]:
data = pd.read_csv("cs_bisnode_panel.csv")
dicionario_de_dados_0 = pd.read_excel("bisnode_variable_names.xls", header=4)

Visualização inicial da base de dados

In [ ]:
data.head()

In [ ]:
data.describe()

## Limpeza dos Dados

### Dicionário de dados - Limpeza

Ao carregar o dicionário de dados a primeira coluna pega seu nome da quarta linha da tabela (argumento `header=4` acima). As outras três colunas são nomeadas abaixo.

In [ ]:
dicionario_de_dados_1 = dicionario_de_dados_0.rename({'Unnamed: 1': 'description',
                                                      'Unnamed: 2': 'type',
                                                      'Unnamed: 3': 'footnote'},
                                                     axis=1)

In [ ]:
dicionario_de_dados_1.columns

Então, retiramos as linhas não relevantes para a análise, incluindo linhas totalmente em branco e uma linha com informação de versão da base de dados: 
 - `v 0.92. 2021-02-04`

In [ ]:
dicionario_de_dados = dicionario_de_dados_1\
                       .drop(index=54)\
                       .dropna(how="all")\
                       .reset_index()\
                       .drop('index', axis='columns')

In [ ]:
dicionario_de_dados.sample(5)

## Dados
### Removendo colunas específicas:

In [ ]:
columns_to_remove = ['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D', 'begin', 'end'] # além das variáveis da questões, removido begin e end, pois utilizaremos apenas o year 

data.drop(columns=columns_to_remove, inplace=True)

data.columns

### Removendo dados do ano 2016:
---

Registros do ano de 2016 são removidos do conjunto.

In [ ]:
data = data[data["year"]!=2016].copy()
data["year"].unique()

Dados destas colunas precisam estar em formato de datetime para serem operados corretamente, então vamos converte-los.

In [ ]:
colunas_data = ["founded_date", "exit_date"]

data.dtypes[colunas_data]

In [ ]:
for column in colunas_data:
    data[column] = pd.to_datetime(data[column], format="%Y-%m-%d")

data[colunas_data].dtypes

### Missing data:
---

Vamos verificar dados faltando do banco de dados:

Criação de função para analisar os dados faltantes.

In [ ]:
def show_missing(df):
    """Return a Pandas dataframe describing the contents of a source dataframe including missing values."""
    
    variables = []
    dtypes = []
    count = []
    unique = []
    missing = []
    pc_missing = []
    
    for item in df.columns:
        variables.append(item)
        dtypes.append(df[item].dtype)
        count.append(len(df[item]))
        unique.append(len(df[item].unique()))
        missing.append(df[item].isna().sum())
        pc_missing.append(round((df[item].isna().sum() / len(df[item])) * 100, 2))

    output = pd.DataFrame({
        'variable': variables, 
        'dtype': dtypes,
        'count': count,
        'unique': unique,
        'missing': missing, 
        'pc_missing': pc_missing
    })    
        
    return output

Verificando dados com maior falta de informações:

In [ ]:
missing_data = show_missing(data).sort_values("pc_missing", ascending=False, ignore_index = True)

index_full_data = list(missing_data[missing_data["missing"]==0].index)

missing_data.drop(labels=index_full_data, axis="index", inplace=True)

missing_data

In [ ]:
missing_val_columns = missing_data["variable"][missing_data["pc_missing"]>0]

In [ ]:
msno.bar(data[missing_val_columns], figsize=(16, 4))

## Tratamento de dados ausentes

- exit_year(86.03%) e exit_date (79.80%): Será utilizado apenas o exit_year para saber se a empresa estava ativa no ano de análise. Quando o exit_year estiver ausente, vamos procurar se tem valor na exit_date para pegar o ano de encerramento da empresa. Em seguida, vamos excluir a variével exit_date.

In [ ]:
# Quando o exit_year estiver ausente, pegue o ano de exit_date
data.loc[data['exit_year'].isna(), 'exit_year'] = pd.to_datetime(data['exit_date']).dt.year

# Substituir valores NaN em exit_year por "-"
data['exit_year'] = data['exit_year'].fillna('-')

# Excluir a coluna exit_date
data = data.drop(columns='exit_date')

### 1. Tratamento de dados com menos de 4% de dados ausentes

- founded_year (20.2%): essa variável pode ser obtida a partir da variável "founded_date". Como a variável "founded_date" apenas 0.02% de dados ausente, a found_year será formada a partir do ano da founded_year e depois vamos excluir a coluna founded_data

In [ ]:
# Removendo linhas onde 'founded_date' é NaN
data = data.dropna(subset=['founded_date'])

# Extraindo o ano de 'founded_date' e substituindo os valores ausentes em 'founded_year'
data['founded_year'] = data['founded_date'].dt.year

# Descartanto a coluna 'founded_date', pois utilizar apemas o ano de fundação da empresa é suficiente
data = data.drop(columns=['founded_date'])

Analisando percentuais de missing após os tratamentos.

In [ ]:
missing_data2 = show_missing(data).sort_values("pc_missing", ascending=False, ignore_index = True)

index_full_data = list(missing_data2[missing_data2["missing"]==0].index)

missing_data2.drop(labels=index_full_data, axis="index", inplace=True)

missing_data2

- todos os dados com <4% de dados ausentes: Vamos iniciar tratando os dados que estejam com menos de 4% de dados ausentes. Primeiramente, serão analisados se as linhas das colunas com menos de 4% de dados ausentes coincidem ou não.

In [ ]:
# Lista de colunas com menos de 4% de dados ausentes

cols = list(missing_data2["variable"][missing_data2["pc_missing"]<5.0])

# Verificar quais linhas possuem dados ausentes nessas colunas

missing_rows = data[cols].isnull().any(axis=1)      

# Calcular o percentual de linhas com dados ausentes

pc_missing_rows = 100 * missing_rows.sum() / len(data)

print(f"Percentual de linhas com dados ausentes em pelo menos uma das colunas mencionadas: {pc_missing_rows:.2f}%")

In [ ]:
# Removendo variáveis com menos de 5% de dados ausentes
removable_na_columns = list(missing_data2["variable"][missing_data2["pc_missing"]<5.0])

data.dropna(subset=removable_na_columns, ignore_index=True, inplace=True)

Observando os valores de missing após os tratamentos.

In [ ]:
missing_data_3 = show_missing(data).sort_values("pc_missing", ascending=False, ignore_index = True)

index_full_data = list(missing_data_3[missing_data_3["missing"]==0].index)

missing_data_3.drop(labels=index_full_data, axis="index", inplace=True)

missing_data_3

Analisando dados ausentes

In [ ]:
msno.matrix(data)

Analisando variáveis labor_avg e birth_year

In [ ]:
 # Plotando um histograma para 'labor_avg'
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(data['labor_avg'].dropna(), bins=30, color='blue', alpha=0.7)
plt.title('Distribuição de labor_avg')
plt.xlabel('labor_avg')
plt.ylabel('Frequência')

# Plotando um histograma para 'birth year CEO'
plt.subplot(1, 2, 2)
plt.hist(data['birth_year'].dropna(), bins=30, color='green', alpha=0.7)
plt.title('Distribuição de birth year CEO')
plt.xlabel('birth year CEO')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()

In [ ]:
# Calculando e mostrando a skewness (assimetria) das variáveis
print(f"Assimetria de labor_avg: {data['labor_avg'].skew()}")
print(f"Assimetria de birth year CEO: {data['birth_year'].skew()}")

In [ ]:
# Configurando o estilo do Seaborn
sns.set(style="whitegrid")

plt.figure(figsize=(12, 6))

# Boxplot para 'labor_avg'
plt.subplot(1, 2, 1)
sns.boxplot(y=data['labor_avg'], color='blue')
plt.title('Boxplot de labor_avg')

# Boxplot para 'birth year CEO'
plt.subplot(1, 2, 2)
sns.boxplot(y=data['birth_year'], color='green')
plt.title('Boxplot de birth year CEO')

plt.tight_layout()
plt.show()

## Tratar demais variáveis com MICE

Os demais valores ausentes serão preenchidos utilizando o método MICE (Multiple Imputation by Chained Equations) com a biblioteca 'IterativeImputer' do sci

In [ ]:
df_mice = data.copy()
df_mice.head()

In [ ]:
# Cópia do df_mice para converter em dummies
df_mice2 = df_mice.copy()
df_mice2 = pd.get_dummies(df_mice2)

In [ ]:
# Define Input de MICE e preenche NaN 

mice_imputer = IterativeImputer(estimator=linear_model.BayesianRidge(), n_nearest_features=None, imputation_order='ascending')
df_mice_imputed = pd.DataFrame(mice_imputer.fit_transform(df_mice2), columns=df_mice2.columns)

In [ ]:
df_mice_imputed

In [ ]:
missing_data_3 = show_missing(df_mice_imputed).sort_values("pc_missing", ascending=False, ignore_index = True)

index_full_data_3 = list(missing_data_3[missing_data_3["missing"]==0].index)

missing_data_3.drop(labels=index_full_data_3, axis="index", inplace=True)

missing_data_3

In [ ]:
# Verificando dados imputados 

#MICE imputacao
fig = plt.Figure()
null_values = data['labor_avg'].isnull() 
fig = df_mice_imputed.plot(x='sales', y='labor_avg', kind='scatter',
                           c=null_values, cmap='winter', s = 15,
                           title='MICE Imputation', colorbar=False)

In [ ]:
data.columns

In [ ]:
df_mice_imputed.columns

Reverter o processo de pd.get_dummies com base no df_mice_imputed

In [ ]:
# Dicionário que contém as colunas originais e suas respectivas colunas transformadas em dummies
categorical_cols_transformed = {
    'exit_year': [col for col in df_mice_imputed.columns if 'exit_year' in col],
    'gender': [col for col in df_mice_imputed.columns if 'gender_' in col],
    'origin': [col for col in df_mice_imputed.columns if 'origin_' in col],
    'region_m': [col for col in df_mice_imputed.columns if 'region_m_' in col]
}

# Para cada coluna original, encontrar a coluna dummy com o valor mais alto (1) e restaurar a coluna original
for original_col, dummies in categorical_cols_transformed.items():
    df_mice_imputed[original_col] = df_mice_imputed[dummies].idxmax(axis=1).str.replace(original_col + "_", "")
    df_mice_imputed.drop(dummies, axis=1, inplace=True)

In [ ]:
df_mice_imputed.columns

Vemos que temos dados de venda com erros, onde há valores negativos para vendas, vamos tratar estes dados substituindo valores negativos por nulos (valor = 0)

In [ ]:
# Remodelando os dados de vendas para corrigir os erros:

for x in ["sales"]:
    df_mice_imputed[x] = np.where(df_mice_imputed[x]<0, 0, df_mice_imputed[x])

df_mice_imputed['fechado'] = condition.astype(int)

### Criando coluna para Variável Dependente:
---

- Vamos operar com o conceito de "atividade", empresas que não tiveram atividade por mais de 2 anos são consideradas "inativas".

Obs.: ativa = 0, inativa = 1

In [ ]:
# Sort the DataFrame by company and year
df_mice_imputed.sort_values(by=['comp_id', 'year'], inplace=True)

# Create shifted columns to check sales in the following 2 years
conditions_x1 = [((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-1)) &
                  (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-1) - 1)),

                 ((df_mice_imputed['comp_id'] != df_mice_imputed['comp_id'].shift(-1)) |
                  df_mice_imputed['year'] != df_mice_imputed['year'].shift(-1))]

values_x1 = [df_mice_imputed['sales'].shift(-1),
             np.nan]

df_mice_imputed['sales_x1'] = pd.Series(np.select(conditions_x1, values_x1)).fillna(0)


conditions_x2 = [
    ((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-1)) &   
     (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-1) - 2)),

    ((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-2)) &
     (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-2) - 2)),

    True
]

values_x2 = [df_mice_imputed['sales'].shift(-1),
             df_mice_imputed['sales'].shift(-2),
             np.nan]

df_mice_imputed['sales_x2'] = pd.Series(np.select(conditions_x2, values_x2)).fillna(0)


# Create a condition to identify companies that ceased to operate 
# (sem vendas por mais de 2 anos)
condition = ((df_mice_imputed['sales_x1'] == 0) & (df_mice_imputed['sales_x2'] == 0)) 

# Create a new 'dependente' column with 1 for ceased companies and 0 otherwise
df_mice_imputed['fechado'] = condition.astype(int)

Vamos conferir os valores da variavel dependente para algumas empresas do dataframe:

In [ ]:
# lista de colunas para avaliação:
check_list = ["comp_id", "year", "exit_year", "sales", "sales_x1", "sales_x2", "dependente"]

filtro = df_mice_imputed[check_list]
filtro_sub = filtro[(filtro["comp_id"] == 464021159936) | (filtro["comp_id"] == 1001541)]

filtro_sub

In [ ]:
df_mice_imputed[check_list].describe()

Corrigimos os valores de vendas negativas, e assim também de dependentes que pudessem estar sendo afetados.

Vamos agora:
 
- tratar por fim os casos em que não há informações de venda para um próximo ano (por não haver um próximo ano;

- criar uma coluna de vendas em "Log" para tratar a assimetria dos dados.

Vamos averiguar agora os anos em que as empresas tiveram atividade e inatividade:

In [ ]:
# Convertendo 'comp_id' para float antes de agrupar
df_mice_imputed['comp_id'] = df_mice_imputed['comp_id'].astype(float)

data_grouped = df_mice_imputed.groupby('comp_id')

#Contando anos de acompanhamento
comp_years = data_grouped['year'].count()

#Contando anos com vendas
sales_years = data_grouped.apply(lambda group: (group['sales'] > 0).sum())

#contando anos sem vendas
no_sales_years = data_grouped.apply(lambda group: (group['sales'] == 0).sum())

#Contando anos de "inatividade"
inative_years = data_grouped.apply(lambda group: (group['dependente'] == 1).sum())

In [ ]:
pd.options.display.float_format = '{:.1f}'.format

activity_df = pd.DataFrame({"Total years":comp_years,
                            "Sales years":sales_years, 
                            "No sales years":no_sales_years,
                            "Inative years":inative_years}).reset_index()

activity_df['comp_id'] = activity_df['comp_id'].astype(float)
activity_df

## 2012

In [ ]:
# Criando dataframe apenas com os dados do ano de 2012

data_2012 = df_mice_imputed[df_mice_imputed['year'] == 2012].copy()
data_2012.year.unique() # verificando coluna de ano

In [ ]:
# considerando apenas sales > 1000 e < 10000000

data_2012 = data_2012[(data_2012['sales'] > 1000) & (data_2012['sales'] < 10000000) ]

In [ ]:
# Verificando valores da coluna year_exit

data_2012['exit_year'].value_counts()

In [ ]:
# check list
check_list = ["comp_id", "year", "exit_year", "sales", "sales_x1", "sales_x2", "dependente"]

# Converta "-" para NaN
data_2012["exit_year"].replace("-", np.nan, inplace=True)

# Converta a coluna para float
data_2012["exit_year"] = data_2012["exit_year"].astype(float)

# Filtre o dataframe
filtro = data_2012[check_list]
filtro_sub = filtro[filtro["exit_year"] < 2012]
filtro_sub

Conforme apresentado acima, algumas empresas que encerraram suas atividades antes de 2012 apresentaram vendas em 2012 e nos anos seguintes. Entretanto, na nossa análise, verificamos se a empresa está ativa em 2012 com base na variável "exit_year". Essas empresas podem terem sido reabertas ou pode ser um erro na base, mas, como são poucos casos, vamos excluir essas empresas que encerraram a atividade antes de 2012.

Verificando as empresas que encerraram as atividades em 2012. 

In [ ]:
# check list
check_list = ["comp_id", "year", "exit_year", "sales", "sales_x1", "sales_x2", "dependente"]

# Converta "-" para NaN
data_2012["exit_year"].replace("-", np.nan, inplace=True)

# Converta a coluna para float
data_2012["exit_year"] = data_2012["exit_year"].astype(float)

# Filtre o dataframe
filtro = data_2012[check_list]
filtro_sub = filtro[filtro["exit_year"] == 2012]
filtro_sub

In [ ]:
# Somar as colunas sales_x1 e sales_x2
total_sales_x1 = filtro_sub['sales_x1'].sum()
total_sales_x2 = filtro_sub['sales_x2'].sum()

print(f"Total de sales_x1: {total_sales_x1}")
print(f"Total de sales_x2: {total_sales_x2}")

Algumas empresas que encerraram as atividades até 31/12/2012 tiveram sales durante o ano de 2012, mas não tiveram vendas no ano seguinte, conforme o esperado. Como nosso objetivo é prever as empresas que estavam ativas em 31/12/2012 e tiveram suas atividades encerradas em até dois anos, as empresas que encerraram as atividades em 2012 serão retiradas da base também. 

In [ ]:
# Retirando empresas que encerraram a atitidade até 2012, portanto não estavam ativa na nossa data base de análise
data_2012 = data_2012.copy()
data_2012 = data_2012[data_2012['exit_year'] > 2012]

# Verificando valores da coluna year_exit
data_2012['exit_year'].value_counts()

In [ ]:
# Filtrar apenas colunas numéricas para calcular skewness
numeric_cols = data_2012.select_dtypes(include=[np.number]).columns # Avaliar a simetria das variáveis

# Avaliar a simetria das colunas numéricas
skewness_values = data_2012[numeric_cols].skew()

# Ordenando os valores de skewness do maior para o menor
sorted_skewness_values = skewness_values.sort_values(ascending=False)

# Mostrando os valores de skewness ordenados
print(sorted_skewness_values)

In [ ]:
# Identificando colunas com skewness significativamente diferente de zero
cols_to_transform = skewness_values[skewness_values.abs() > 0.5].index
print(cols_to_transform)

Nas estatísticas descritivas abaixo, observa-se que a média é maior que o terceiro quartil, indicando uma distribuição bastante assimétrica. Criamos então uma com o logarítmo de `sales` para auxiliar na análise.

In [ ]:
print(data_2012['sales'].describe())

A coluna log_sales mostra que, embora muitas empresas tenham vendas relativamente baixas (indicado por um logaritmo de vendas próximo ou igual a zero), há empresas que se destacam com vendas substancialmente mais altas. A transformação logarítmica ajudou a reduzir a assimetria e a concentrar os dados, tornando-os mais tratáveis para análise estatística e modelagem. No entanto, a presença de uma assimetria à esquerda ainda é evidente, sugerindo que muitas empresas no dataset têm vendas baixas ou nulas.

In [ ]:
# Transformando a coluna sales usando log
data_2012.loc[:,'log_sales'] = data_2012.sales\
                                        .apply(lambda x: math.log(x)\
                                               if x != 0\
                                               else 0)

# Mostrando estatísticas descritivas da coluna log_sales
print(data_2012.log_sales.describe())

Comparando a distribuição da variável sales e log_sales.

In [ ]:
# Criando histogramas
plt.figure(figsize=(12, 6))

# Histograma de sales
plt.subplot(1, 2, 1)
plt.hist(data_2012['sales'], bins=50, color='blue', edgecolor='black')
plt.title('Histograma de Sales')
plt.xlabel('Sales')
plt.ylabel('Frequência')

# Histograma de log_sales
plt.subplot(1, 2, 2)
plt.hist(data_2012['log_sales'], bins=50, color='green', edgecolor='black')
plt.title('Histograma de Log de Sales')
plt.xlabel('Log Sales')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()


### Criação de variáveis

In [ ]:
# 1. Idade da Empresa:
data_2012['company_age'] = 2012 - data_2012['founded_year']

In [ ]:
# 2. Alavancagem Financeira:
#data_2012['financial_leverage'] = data_2012['curr_liab'] / data_2012['share_eq']

In [ ]:
# 3. Liquidez:
#data_2012['liquidity_ratio'] = data_2012['liq_assets'] / data_2012['curr_liab']

In [ ]:
# 4. Eficiência:
#data_2012['efficiency'] = data_2012['sales'] / data_2012['labor_avg']

In [ ]:
# 5. Tamanho da Empresa (exemplo baseado em vendas):
#sales_bins = [0, 1e6, 1e9, float('inf')]  # Exemplo de categorias: <1M, 1M-1B, >1B
#labels = ['small', 'medium', 'large']
#data_2012['company_size'] = pd.cut(data_2012['sales'], bins=sales_bins, labels=labels, right=False)

In [ ]:
# 5. Margem de Lucro:
#data_2012['profit_margin'] = data_2012['profit_loss_year'] / data_2012['sales']

In [ ]:
data_2012 = data_2012.drop(columns=['sales_x1','sales_x2', 'exit_year', 'year'])

## Matriz de correlação

In [ ]:
# Listar todas as colunas exceto "dependente"
cols = [col for col in data_2012.columns if col != "dependente"]

# Dividir as colunas em dois grupos
half = len(cols) // 2
group1 = cols[:half] + ["dependente"]
group2 = cols[half:] + ["dependente"]

# Considerar apenas colunas numéricas
group1 = [col for col in group2 if data_2012[col].dtype in ['int64', 'float64']]
group2 = [col for col in group2 if data_2012[col].dtype in ['int64', 'float64']]

# Criar as duas matrizes de correlação
correlation_matrix1 = data_2012[group1].corr()
correlation_matrix2 = data_2012[group2].corr()

# Visualizar a primeira matriz de correlação
plt.figure(figsize=(16,10))
sns.heatmap(correlation_matrix1, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix - Group 1')
plt.show()

# Visualizar a segunda matriz de correlação
plt.figure(figsize=(16,10))
sns.heatmap(correlation_matrix2, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix - Group 2')
plt.show()

missing_data_4 = show_missing(data_2012).sort_values("pc_missing", ascending=False, ignore_index = True)

full_index = list(missing_data_3[missing_data_3["missing"]==0].index)

missing_data_4.drop(labels=full_index, axis="index", inplace=True)

missing_data_4


In [ ]:
data_2012.to_csv("data_2012.csv", index=False)

In [ ]:
data_2012.columns

In [ ]:
#data_2012.to_excel("data_2012.xlsx")